| version | commentaire |
| --- | --- |
| v0r1 | version initiale |
| v0rN |  |


# Forum des métiers - Répartition de groupes et allocation de slots

## Introduction

Ce notebook permet d'effectuer des allocations individuelles à des slots de visite de représentants des métiers

Contraintes :

- pas d'élève sans affectation de métier pour un créneau (timeslot) donné,
- le moins possible de métier (job) sans affectation de groupe pour un timeslot donné,
- nombre de souhaits par élève =< timeslots,
- il peut y avoir n représentants pour un métier donné.

=> le nombre de métiers doit être plus grand que le nombre de groupes d'élèves pour que chaque groupe d'élèves visite un métier pendant un créneau mais pas trop non plus pour minimiser les métiers sans groupe affecté pendant un créneau.


La conversion du contenu d'un fichier Excel au formal JSON peut être effectuée sur le site suivant :
https://tableconvert.com/excel-to-json

## Importations des modules

In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import math
import random
from json import loads, dumps
from numpy.random import default_rng
from typing import TypeAlias
from typing import Dict, Tuple, Sequence, List

In [2]:
from dataclasses import dataclass

In [4]:
from collections import defaultdict

In [5]:
Wishes : TypeAlias = List[int]
Id : TypeAlias = int
StudentId : TypeAlias = int

# { 'id' : int, 'wishes':list[int], 'itv_visited':list[(int,int)] }
Student : TypeAlias = dict[str,any]

# liste des intervenants pour un même job
# [ { 'id':int, 'batch_size':int, 'Q':list[int] } ]
Job_Intervenants : TypeAlias = list[dict[str,any]] 

# liste d'intervenants par job
Q_Intervenants : TypeAlias = list[Job_Intervenants]

# list timetable
Job : TypeAlias = int
Itv : TypeAlias = int
TimeslotStudentVisit : TypeAlias = Tuple[StudentId, Tuple[Job,Itv]]
TimetableSlot : TypeAlias = list[TimeslotStudentVisit]
Timetable : TypeAlias = list[TimetableSlot]


In [6]:
@dataclass
class CStudent:
    id:Id
    wishes:Wishes
    itv_visits:List[int]

In [7]:
@dataclass
class CIntervenants:
    id:int
    batch_size:int
    Q:list[int]

In [8]:
s=CStudent(3000,[1,2,3],[])
type(s)

__main__.CStudent

In [9]:
s = { 'id' : 3000, 'w':[] }
type(s)

dict

In [10]:
# importation d'un échantillon de données pour les tests unitaires
sample_class_32 = pd.read_json('sample-3e2-new.json')
result = sample_class_32.to_json(orient="index")
parsed = loads(result)
sample_class_32 = list(parsed.values())


In [11]:
type(sample_class_32[0])

dict

In [12]:
intervenants_data = pd.read_json('intervenants_data_sample.json')

In [13]:
intervenants_data

,0,1,2,3,4,5
0,None,None,None,None,None,None
1,"{'id': 10, 'batch_size': 4}","{'id': 11, 'batch_size': 4}","{'id': 12, 'batch_size': 4}",None,None,None
2,"{'id': 20, 'batch_size': 4}","{'id': 21, 'batch_size': 4}",None,None,None,None
3,"{'id': 30, 'batch_size': 4}","{'id': 31, 'batch_size': 4}",None,None,None,None
4,"{'id': 40, 'batch_size': 4}",None,None,None,None,None
5,"{'id': 50, 'batch_size': 4}","{'id': 51, 'batch_size': 4}","{'id': 52, 'batch_size': 4}","{'id': 53, 'batch_size': 4}","{'id': 54, 'batch_size': 4}","{'id': 55, 'batch_size': 4}"


In [14]:
NB_JOBS = intervenants_data.shape[0]
NB_JOBS

6

In [15]:
nb_intervenants_per_job = [ np.argwhere(intervenants != None).size for intervenants in intervenants_data[:].to_numpy()]
print("nb_intervenants_per_job : {}".format(nb_intervenants_per_job))

itv_id_per_job = [ [ itv["id"] for itv in intervenants if itv != None ] for intervenants in intervenants_data[:].to_numpy() ]
print("itv_id_per_job : {}".format(itv_id_per_job))

batch_size_per_itv_per_job = [ [ itv["batch_size"] for itv in intervenants if itv != None ] for intervenants in intervenants_data[:].to_numpy() ]
print("batch_size_per_itv_per_job : {}".format(batch_size_per_itv_per_job))

itv_data_per_job = [ [ itv for itv in intervenants if itv != None ] for intervenants in intervenants_data[:].to_numpy() ]
print("itv_data_per_job : {}".format(itv_data_per_job))

nb_intervenants_per_job : [0, 3, 2, 2, 1, 6]
itv_id_per_job : [[], [10, 11, 12], [20, 21], [30, 31], [40], [50, 51, 52, 53, 54, 55]]
batch_size_per_itv_per_job : [[], [4, 4, 4], [4, 4], [4, 4], [4], [4, 4, 4, 4, 4, 4]]
itv_data_per_job : [[], [{'id': 10, 'batch_size': 4}, {'id': 11, 'batch_size': 4}, {'id': 12, 'batch_size': 4}], [{'id': 20, 'batch_size': 4}, {'id': 21, 'batch_size': 4}], [{'id': 30, 'batch_size': 4}, {'id': 31, 'batch_size': 4}], [{'id': 40, 'batch_size': 4}], [{'id': 50, 'batch_size': 4}, {'id': 51, 'batch_size': 4}, {'id': 52, 'batch_size': 4}, {'id': 53, 'batch_size': 4}, {'id': 54, 'batch_size': 4}, {'id': 55, 'batch_size': 4}]]


## Fonctions utilitaires

In [16]:
def build_dataframe_labels(jobs_sequence_per_groups):
    """
    Cette fonction établit une liste de labels d'indexes pour présenter les résultats d'affectation de jobs aux groupes via panda.
    La donnée en entrée doit être issue de la fonction : timeslot_job_sequence_per_groups().
    """
    nb_groups, nb_slots = np.array(jobs_sequence_per_groups).shape
    index_labels = [ "group "+str(i) for i in range(0, nb_groups) ]
    col_labels = [ "slot "+ str(c) for c in range(0, nb_slots) ]
    return index_labels, col_labels

In [17]:
# test function

NB_groups=2
GROUP_SIZE=3
NB_JOBS=4
NB_WISHES=3
FIXED_RND_THRESHOLD=10

# Allocations des élèves aux timeslots

Un timeslot est un créneau horaire durant lequel différents batch d'élèves vont visiter un (et un seul) intervenant de métier (job).

Un timeslot est représenté par un vecteur dont chaque index est celui d'un intervenant de métier et la valeur à cet index est une liste d'élèves (batch pour ce slot).
La taille du batch peut être différente pour chaque représentant dans un même timeslot

```
intervenant 0         1    2    3   4   
slot 0 [ [ student ], .. , .., .., .. ]
slot 1 [    ..      , .. , .., .., .. ]
slot 2 [    ..      , .. , .., .., .. ]
  :
```

La fonction d'entrée de l'allocation des timeslots est `allocate_slots`.

## Q-intervenant

Avant de constituer les batch d'un timeslot, on place les élèves dans des files d'attente par intervenant représentant la catégorie de métier.

Un élève est placé dans une file d'attente en fonction de ses préférences de métier (non encore visitées) et par ordre de préférence.

Un élève ne peut être placé dans une file d'attente que si celle-ci n'est pas déjà pleine (pour un intervenant donné).

Si toutes les files d'attente des représentants d'un métier sont pleines, on considère le métier non encore visité suivant de l'élève et on retente une affectation dans une file d'attente correspondante.

Si l'élève ne peut toujours pas être placé dans une file d'attente selon ses préférences métier, on le place dans une file d'attente encore libre d'un intervenant que cet élève n'a pas encore visité.

Exemple :

Pour un élève qui a 5 choix dont le premier a déjà été visité dans un timeslot antérieur, pour la file d'attente (Q-intervenant) du timeslot en constitution, on place l'élève dans la file d'attente d'un intervenant du métier 3 si la file de cet intervenant n'est pas encore pleine (selon BATCH_SIZE[intervenant]).

```
{ "id" : 3100, wishes : [ -1, 3, 13, 8, 5 ]}

Q[<intervenant(3)>].append(3100)
````

Si un métier a plusieurs intervenants, on place l'élève dans la file d'attente la plus courte.

Si toutes les files d'attente des intervenants du métier No 3 sont pleines, on reprend la tentative d'affectation avec le métier 13.

Si malgré tout l'élève ne peut pas être placé dans une file d'attente, on le place dans une file d'attente d'un intervenant que cet élève n'a pas encore visité dans un précédent timeslot. 

In [18]:
def scrap_student_choice(students:list[Student], id:int, job:int)->list[Student]:
    """
    Cette fonction positionne à -1 le choix de métier (job) d'un élève (id) figurant dans une liste d'élèves (students).
    """

    students_ids = [ student["id"] for student in students ]

    l_idx_student = np.argwhere(np.array(students_ids)==id)
    
    if l_idx_student.size == 0:
        return students
    
    idx_student = l_idx_student[0][0]

    l_idx_job_to_scrap = np.argwhere(np.array(students[idx_student]['wishes'])==job)

    if l_idx_job_to_scrap.size == 0 :
        return students
    
    idx_job_to_scrap = l_idx_job_to_scrap[0][0]

    students[idx_student]['wishes'][idx_job_to_scrap] = -1
    return students

In [19]:
def scrap_single_student_choice(student:Student, job:int)->Student:
    l_idx_job_to_scrap = np.argwhere(np.array(student['wishes'])==job)

    if l_idx_job_to_scrap.size == 0 :
        return student
    
    idx_job_to_scrap = l_idx_job_to_scrap[0][0]

    student['wishes'][idx_job_to_scrap] = -1
    return student    

In [20]:
# test
test_students = [ 
    { "id" : 3100, "wishes" : [ 4,3,2,1,0 ] },
    { "id" : 3101, "wishes" : [ 0,1,2,3,4 ] },
    { "id" : 3102, "wishes" : [ 2,1,0,4,3 ] }
]
scrap_student_choice(test_students, 3101, 1)

[{'id': 3100, 'wishes': [4, 3, 2, 1, 0]},
 {'id': 3101, 'wishes': [0, -1, 2, 3, 4]},
 {'id': 3102, 'wishes': [2, 1, 0, 4, 3]}]

In [21]:
def get_next_valid_student_choice(students:list[Student], id:int)->int:
    """ 
    Cette fonction retourne le prochain choix de métier encore valide d'un élève selon l'ordre de priorité
    """
    students_ids = [ student["id"] for student in students ]

    l_idx_student = np.argwhere(np.array(students_ids)==id)
    
    if l_idx_student.size == 0:
        return None

    idx_student = l_idx_student[0][0]
    l_idx_valid_wishes = np.argwhere(np.array(students[idx_student]['wishes'])!=-1)

    if l_idx_valid_wishes.size == 0:
        return None
    
    return students[idx_student]['wishes'][l_idx_valid_wishes[0][0]]


In [22]:
# test
test_students = [ 
    { "id" : 3100, "wishes" : [ 4,3,2,1,0 ] },
    { "id" : 3101, "wishes" : [ 0,1,2,3,4 ] },
    { "id" : 3102, "wishes" : [ 2,1,0,4,3 ] }
]
scrap_student_choice(test_students, 3101, 0)
scrap_student_choice(test_students, 3101, 1)

get_next_valid_student_choice(test_students, 3101)

2

In [23]:
# test
np.array(nb_intervenants_per_job[:]).T.tolist()

[0, 3, 2, 2, 1, 6]

## Fonctions Q_intervenants

In [24]:
def init_q_intervenants(intervenants_data:pd.DataFrame)->Q_Intervenants:
    """
    Cette fonction initialise une structure contenant les files d'attente vides par intervenant et par job
    [   
        # job 0
        [],  

        # job 1
        [   {'id': 10, 'batch_size': 4, 'Q': []},
            {'id': 11, 'batch_size': 4, 'Q': []},
            {'id': 12, 'batch_size': 4, 'Q': []}
        ],

        # job 2
        [   {'id': 20, 'batch_size': 4, 'Q': []}, 
            {'id': 21, 'batch_size': 4, 'Q': []}
        ],
        ...
    ]

    """
    nb_intervenants_per_job     = [ np.argwhere(intervenants != None).size for intervenants in intervenants_data[:].to_numpy()]
    itv_id_per_job              = [ [ itv["id"] for itv in intervenants if itv != None ] for intervenants in intervenants_data[:].to_numpy() ]
    batch_size_per_itv_per_job  = [ [ itv["batch_size"] for itv in intervenants if itv != None ] for intervenants in intervenants_data[:].to_numpy() ]

    nb_jobs = len(nb_intervenants_per_job)

    l_q = []
    for n_j in range(nb_jobs):
        l_q_itv = []
        for n_itv in range(len(itv_id_per_job[n_j])):
            l_q_itv.append({ "id" : itv_id_per_job[n_j][n_itv], "batch_size":batch_size_per_itv_per_job[n_j][n_itv], "Q":[]})
        l_q.append(l_q_itv)
    return l_q

In [25]:
# test init_q_intervenants
init_q_intervenants(intervenants_data)

[[],
 [{'id': 10, 'batch_size': 4, 'Q': []},
  {'id': 11, 'batch_size': 4, 'Q': []},
  {'id': 12, 'batch_size': 4, 'Q': []}],
 [{'id': 20, 'batch_size': 4, 'Q': []}, {'id': 21, 'batch_size': 4, 'Q': []}],
 [{'id': 30, 'batch_size': 4, 'Q': []}, {'id': 31, 'batch_size': 4, 'Q': []}],
 [{'id': 40, 'batch_size': 4, 'Q': []}],
 [{'id': 50, 'batch_size': 4, 'Q': []},
  {'id': 51, 'batch_size': 4, 'Q': []},
  {'id': 52, 'batch_size': 4, 'Q': []},
  {'id': 53, 'batch_size': 4, 'Q': []},
  {'id': 54, 'batch_size': 4, 'Q': []},
  {'id': 55, 'batch_size': 4, 'Q': []}]]

In [26]:
def get_smallest_q_for_job(q_intervenants:Q_Intervenants, job:int)->int:
    q_sizes_for_job = [ np.array(q['Q']).size for q in q_intervenants[job] ]
    return int(np.argmin(np.array(q_sizes_for_job)))

In [27]:
# test
test_q_intervenants = init_q_intervenants(intervenants_data)
get_smallest_q_for_job(test_q_intervenants, 1)

0

In [28]:
def get_batch_size(q_intervenants:Q_Intervenants, job:int, itv:int)->int:
    if job > len(q_intervenants):
        return None
    if itv > len(q_intervenants[job]):
        return None
    
    return q_intervenants[job][itv]['batch_size']

In [29]:
# test : get_batch_size
test_q_intervenants = init_q_intervenants(intervenants_data)
print(get_batch_size(test_q_intervenants, 1, 2))
print(get_batch_size(test_q_intervenants, 61, 2))
print(get_batch_size(test_q_intervenants, 1, 100))

4
None
None


In [30]:
def get_current_q_size(q_intervenants:Q_Intervenants, job:int, itv:int)->int:
    if job > len(q_intervenants):
        return None
    if itv > len(q_intervenants[job]):
        return None
    return len(q_intervenants[job][itv]['Q'])

In [31]:
def is_q_intervenant_not_full(q_intervenants:Q_Intervenants, job:int, itv:int)->bool:
    batch_size = get_batch_size(q_intervenants, job, itv)
    current_q_size = get_current_q_size(q_intervenants, job, itv)
    return current_q_size < batch_size


In [32]:
def get_ordered_avail_q_for_job(q_intervenants:Q_Intervenants, job:int)->list[int]:
    
    if job > len(q_intervenants):
        return None
    
    q_avail = [ is_q_intervenant_not_full(q_intervenants, job, itv) for itv in range(len(q_intervenants[job]))]

    q_avail = np.argwhere(np.array(q_avail) == True).T[0]
    #print(q_avail)

    q_sizes = [ np.array(q_intervenants[job][i]).size for i in range(len(q_intervenants[job]))  ]
    #print(q_sizes)
    idx_sorted_q_sizes = np.argsort(np.array(q_sizes))

    idx_sorted_avail_q = [ e for e in idx_sorted_q_sizes.tolist() if e in q_avail ]
    #print(idx_sorted_avail_q)

    #return result
    return idx_sorted_avail_q

In [33]:
# test
test_q_intervenants = init_q_intervenants(intervenants_data)
get_ordered_avail_q_for_job(test_q_intervenants, 1)

[0, 1, 2]

In [34]:
def push_student_in_q(q_intervenants:Q_Intervenants, job:int, itv:int, student_id:int)->Q_Intervenants:
    q_intervenants[job][itv]['Q'].append(student_id)
    return q_intervenants

In [35]:
# test : push_student_in_q
test_students = [ 
    { "id" : 3100, "wishes" : [ 4,3,2,1,5 ], 'itv_visited' : [] }, 
    { "id" : 3101, "wishes" : [ 5,1,2,3,4 ], 'itv_visited' : []  }, 
    { "id" : 3102, "wishes" : [ 2,1,5,4,3 ], 'itv_visited' : []  }, 
    { "id" : 3103, "wishes" : [ 2,4,5,1,3 ], 'itv_visited' : []  },
    { "id" : 3104, "wishes" : [ 0,4,2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3105, "wishes" : [ 0,1,2,3,4 ], 'itv_visited' : []  }, 
    { "id" : 3106, "wishes" : [ 3,1,0,4,2 ], 'itv_visited' : []  }, 
    { "id" : 3107, "wishes" : [ 2,4,0,1,3 ], 'itv_visited' : []  },
    { "id" : 3108, "wishes" : [ 0,1,2,3,4 ], 'itv_visited' : []  }, 
    { "id" : 3109, "wishes" : [ 2,1,0,4,3 ], 'itv_visited' : []  }, 
    { "id" : 3110, "wishes" : [ 2,4,0,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3111, "wishes" : [ 2,4,0,1,3 ], 'itv_visited' : []  },
    { "id" : 3112, "wishes" : [ 4,2,1,0,5 ], 'itv_visited' : []  }
]
test_q_intervenants = init_q_intervenants(intervenants_data)
push_student_in_q(test_q_intervenants, 1, 1, 3101)
push_student_in_q(test_q_intervenants, 1, 1, 3102)
push_student_in_q(test_q_intervenants, 1, 1, 3100)
#push_student_in_q(test_q_intervenants, 1, 1, 3103)
is_q_intervenant_not_full(test_q_intervenants, 1, 1)



True

In [36]:
def build_id_to_job_itv(q_intervenants:Q_Intervenants)->Tuple[int,Tuple[int,int]]:
    """
    [
     [ { <itv-00-data> }, { <itv-01-data> } ],  # intervenants for job 0
     [ { <itv-10-data> } ],                     # intervenants for job 1
     ...
    ] 
    ->
    [ 
        (00 , (0,0)),  # intervenant job 0 - itv 0
        (01 , (0,1)),  # intervenant job 0 - itv 1
        (10 , (1,0)),  # intervenant job 1 - itv 0
        ...
    ]
    """
    result = []    
    nb_jobs = len(q_intervenants)
    for n_job in range(nb_jobs):
        nb_itvs = len(q_intervenants[n_job])
        for n_itv in range(nb_itvs):
            result.append((q_intervenants[n_job][n_itv]['id'], (n_job, n_itv)))
    return result


In [37]:
# test : build_id_to_job_itv
build_id_to_job_itv(init_q_intervenants(intervenants_data))


[(10, (1, 0)),
 (11, (1, 1)),
 (12, (1, 2)),
 (20, (2, 0)),
 (21, (2, 1)),
 (30, (3, 0)),
 (31, (3, 1)),
 (40, (4, 0)),
 (50, (5, 0)),
 (51, (5, 1)),
 (52, (5, 2)),
 (53, (5, 3)),
 (54, (5, 4)),
 (55, (5, 5))]

In [38]:
def get_job_itv_from_itv_id(q_intervenants:Q_Intervenants, itv_id:int)->Tuple[int,int]:
    # obtenir la conversion id (intervenant) -> (job, itv)
    l_id_job_itv = build_id_to_job_itv(q_intervenants)

    # récupérer le (job,itv) qui correspond au itv_id fourni
    itv_id_job_itv = [ (job, itv) for (id,(job,itv)) in l_id_job_itv if id==itv_id]
    if len(itv_id_job_itv)==0:
        #print("get_job_itv_from_itv_id : (None, None) for itv_id {}".format(itv_id))
        return (None,None)
    (job,itv) =  itv_id_job_itv[0] 
    #print("get_job_itv_from_itv_id: {} for itv_id {}".format((job,itv), itv_id))
    return (job,itv)

In [39]:
def push_student_in_q_id(q_intervenants:Q_Intervenants, itv_id:int, student_id:int)->Q_Intervenants:

    # obtenir la conversion id (intervenant) -> (job, itv)
    #l_id_job_itv = build_id_to_job_itv(q_intervenants)

    # récupérer le (job,itv) qui correspond au itv_id fourni
    #itv_id_job_itv = [ (job, itv) for (id,(job,itv)) in l_id_job_itv if id==itv_id]
    #if len(itv_id_job_itv)==0:
    #    return None
    #(job,itv) =  itv_id_job_itv[0]
    
    (job,itv) =  get_job_itv_from_itv_id(q_intervenants, itv_id)

    return push_student_in_q(q_intervenants, job, itv, student_id)

In [40]:
# test : push_student_in_q_id
test_students = [ 
    { "id" : 3100, "wishes" : [ 4,3,2,1,5 ], 'itv_visited' : [] }, 
    { "id" : 3101, "wishes" : [ 5,1,2,3,4 ], 'itv_visited' : []  } ]

test_q_intervenants = init_q_intervenants(intervenants_data)
push_student_in_q_id(test_q_intervenants, 31, 3100)
push_student_in_q_id(test_q_intervenants, 31, 3101)

[[],
 [{'id': 10, 'batch_size': 4, 'Q': []},
  {'id': 11, 'batch_size': 4, 'Q': []},
  {'id': 12, 'batch_size': 4, 'Q': []}],
 [{'id': 20, 'batch_size': 4, 'Q': []}, {'id': 21, 'batch_size': 4, 'Q': []}],
 [{'id': 30, 'batch_size': 4, 'Q': []},
  {'id': 31, 'batch_size': 4, 'Q': [3100, 3101]}],
 [{'id': 40, 'batch_size': 4, 'Q': []}],
 [{'id': 50, 'batch_size': 4, 'Q': []},
  {'id': 51, 'batch_size': 4, 'Q': []},
  {'id': 52, 'batch_size': 4, 'Q': []},
  {'id': 53, 'batch_size': 4, 'Q': []},
  {'id': 54, 'batch_size': 4, 'Q': []},
  {'id': 55, 'batch_size': 4, 'Q': []}]]

In [41]:
def get_next_avail_q(q_intervenants:Q_Intervenants, student:Student, job:int)->int:
    """
    Get a next available Q_intervenant for a particular student and a particular job
    """
    q_avail = get_ordered_avail_q_for_job(q_intervenants, job)

    q_visited = [ q for (_,(jb,q)) in student['itv_visited'] if jb == job ]

    q_avail_not_visited = [ q for q in q_avail if q not in q_visited ]

    next_q = q_avail_not_visited[0] if len(q_avail_not_visited) > 0 else None
    return next_q

In [42]:
# test function : get_ordered_avail_q_for_job
test_students = [ 
    { "id" : 3100, "wishes" : [ 4,3,2,1,5 ], 'itv_visited' : [] }, 
    { "id" : 3101, "wishes" : [ 5,1,2,3,4 ], 'itv_visited' : []  }, 
    { "id" : 3102, "wishes" : [ 2,1,5,4,3 ], 'itv_visited' : []  }, 
    { "id" : 3103, "wishes" : [ 2,4,5,1,3 ], 'itv_visited' : []  },
    { "id" : 3104, "wishes" : [ 0,4,2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3105, "wishes" : [ 0,1,2,3,4 ], 'itv_visited' : []  }, 
    { "id" : 3106, "wishes" : [ 3,1,0,4,2 ], 'itv_visited' : []  }, 
    { "id" : 3107, "wishes" : [ 2,4,0,1,3 ], 'itv_visited' : []  },
    { "id" : 3108, "wishes" : [ 0,1,2,3,4 ], 'itv_visited' : []  }, 
    { "id" : 3109, "wishes" : [ 2,1,0,4,3 ], 'itv_visited' : []  }, 
    { "id" : 3110, "wishes" : [ 2,4,0,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3111, "wishes" : [ 2,4,0,1,3 ], 'itv_visited' : []  },
    { "id" : 3112, "wishes" : [ 4,2,1,0,5 ], 'itv_visited' : []  }
]


test_q_intervenants = init_q_intervenants(intervenants_data)

for n in range(13) :
    next_q = get_next_avail_q(test_q_intervenants, test_students[n], test_students[n]['wishes'][0])

    if next_q != None:
        push_student_in_q(test_q_intervenants, test_students[n]['wishes'][0], next_q, 3100+n)
    else:
        print("eleve {} not pushed".format(3100+n))

print(test_q_intervenants)


eleve 3104 not pushed
eleve 3105 not pushed
eleve 3108 not pushed
[[], [{'id': 10, 'batch_size': 4, 'Q': []}, {'id': 11, 'batch_size': 4, 'Q': []}, {'id': 12, 'batch_size': 4, 'Q': []}], [{'id': 20, 'batch_size': 4, 'Q': [3102, 3103, 3107, 3109]}, {'id': 21, 'batch_size': 4, 'Q': [3110, 3111]}], [{'id': 30, 'batch_size': 4, 'Q': [3106]}, {'id': 31, 'batch_size': 4, 'Q': []}], [{'id': 40, 'batch_size': 4, 'Q': [3100, 3112]}], [{'id': 50, 'batch_size': 4, 'Q': [3101]}, {'id': 51, 'batch_size': 4, 'Q': []}, {'id': 52, 'batch_size': 4, 'Q': []}, {'id': 53, 'batch_size': 4, 'Q': []}, {'id': 54, 'batch_size': 4, 'Q': []}, {'id': 55, 'batch_size': 4, 'Q': []}]]


In [43]:
def get_next_q_to_load(q_intervenants:Q_Intervenants)->list[Tuple[Id, float]]:
    l_ratio_Qs = [ (itv['id'], round(len(itv['Q'])/itv['batch_size'],2)  ) for job_itv in q_intervenants for itv in job_itv  ]
    return l_ratio_Qs

def get_next_q_to_load_sorted(q_intervenants:Q_Intervenants)->list[int]:
    q_id_load_ratio = get_next_q_to_load(q_intervenants)
    q_load_ratio = [ ratio for (_, ratio) in q_id_load_ratio ]
    q_load_ratio_idx = np.argsort(np.array(q_load_ratio))

    return [ q_id_load_ratio[i] for i in q_load_ratio_idx if q_id_load_ratio[i][1] < 1]

In [44]:
# test : get_next_q_to_load + get_next_q_to_load_sorted

test_intervenants_data = pd.DataFrame([
  [],
  [
    { "id" : 10, "batch_size":3 },
    { "id" : 11, "batch_size":3 }
  ],
    [
    { "id" : 20, "batch_size":3 },
    { "id" : 21, "batch_size":3 }
  ],
  [
    { "id" : 30, "batch_size":2 }  
  ]
])
test_q_intervenants = init_q_intervenants(test_intervenants_data)

push_student_in_q_id(test_q_intervenants, 30, 3100)
push_student_in_q_id(test_q_intervenants, 20, 3101)
push_student_in_q_id(test_q_intervenants, 21, 3102)
push_student_in_q_id(test_q_intervenants, 21, 3103)
push_student_in_q_id(test_q_intervenants, 30, 3103)
push_student_in_q_id(test_q_intervenants, 30, 3103)
print(get_next_q_to_load(test_q_intervenants))
get_next_q_to_load_sorted(test_q_intervenants)
#push_student_in_q_id(test_q_intervenants, 30, 3103)
#push_student_in_q_id(test_q_intervenants, 30, 3103)




[(10, 0.0), (11, 0.0), (20, 0.33), (21, 0.67), (30, 1.5)]


[(10, 0.0), (11, 0.0), (20, 0.33), (21, 0.67)]

In [45]:
def convert_to_student_timetable(nb_slots:int, students:list[Student])->Timetable:
    """
    -> [ 
        [ (id, (j,i)), ... ], # slot 0
        [ (id, (j,i)), ... ], # slot 1
        ...
      ]
    """
    timetable = []
    for n in range(nb_slots):
        d = []
        for student in students:
            d.append((student['id'], (student['itv_visited'][n])  if n < len(student['itv_visited']) else None))
        timetable.append(d)
    return timetable

In [90]:
def timetable_student_view(timetable:Timetable)->list[Tuple[StudentId,list[Tuple[int, Tuple[Job,int]]]]]:
    """ 
    -> [ (id, [(s,(j,i))]),... ]
    """
    d_result = defaultdict(list)

    for timeslot in timetable:
        for student_slot_itv_visit in timeslot:
            #(std_id, visit)= student_slot_itv_visit
            #d_result[std_id].append(visit)
            (std_id, (slot,(j,i)))= student_slot_itv_visit
            d_result[std_id].append("{}-{}".format(j,i))


    print(d_result)
    return d_result

## Queing student choice driven

In [47]:
def q_single_student(q_intervenants:Q_Intervenants, student_id:StudentId, job:Job, itv_q:int)->Q_Intervenants:
    return push_student_in_q(q_intervenants, job, itv_q, student_id)

In [48]:
def get_copy_students(students:list[Student])->list[Student]:
    return list([ { 'id':s['id'], 'wishes':list(s['wishes']), 'itv_visited':list(s['itv_visited']) } for s in students ])

In [110]:
def q_students_pass1(n_slot:int, q_intervenants:Q_Intervenants, students:list[Student])->Tuple[Q_Intervenants, list[Student]]:

    result_students = get_copy_students(students)

    #result_students = [ { **student, **{ 'bck_wishes': list(student['wishes']) }}  for student in result_students ]

    remain_students = result_students
    
    while(len(remain_students)>0):
        tmp_students = remain_students
        remain_students = []

        for student in tmp_students:
            # get next job
            next_job = get_next_valid_student_choice(tmp_students, student['id'])
            if next_job == None :
                pass
            else:
                # q available ?
                next_q = get_next_avail_q(q_intervenants, student, next_job)

                student = scrap_single_student_choice(student, next_job)
                if next_q != None:
                    q_intervenants = q_single_student(q_intervenants, student['id'], next_job, next_q)
                    student['itv_visited'].append((n_slot, (next_job,next_q)))
                else:
                    remain_students.append(student)

    result_students = list([ { 'id':s1['id'], 'wishes':list(s1['wishes']), 'itv_visited':list(s2['itv_visited']) } for s1, s2 in zip(students,result_students) ])
    for student in result_students:
        l_job_visited = [ j for (_, (j, _)) in student['itv_visited']]
        for j in l_job_visited:
            student = scrap_single_student_choice(student, j)

        # if student did not get a job visit for this slot, insert a (slot, None) visit in student data
        l_slot_fullfilled = [ slot for (slot, _) in student['itv_visited'] ]
        if not n_slot in l_slot_fullfilled:
            student['itv_visited'].append((n_slot, (-1,0)))

    
    return (q_intervenants,result_students)

In [50]:
def get_timetable_student_driven(nb_slots:int, students:list[Student], intervenants_data:pd.DataFrame)->Timetable:
    result_qs = []
    for slot in range(nb_slots):
        q_intervenants = init_q_intervenants(intervenants_data)
        result_q, students = q_students_pass1(slot, q_intervenants, students)
        result_qs.append(result_q)
    
    #timetable = []
    #for n in range(nb_slots):
    #    d = []
    #    for student in students:
    #        d.append([ student['id'], (student['itv_visited'][n][1],student['itv_visited'][n][2])  if n < len(student['itv_visited']) else None])
    #    timetable.append(d)
    
    return convert_to_student_timetable(nb_slots, students), students

In [51]:
# test : q_students_pass1
test_students = [ 
    { "id" : 3100, "wishes" : [ 2,1,3,4,5 ], 'itv_visited' : [] }, 
    { "id" : 3101, "wishes" : [ 2,1,3,5,4 ], 'itv_visited' : []  }, 
    { "id" : 3102, "wishes" : [ 2,1,5,4,3 ], 'itv_visited' : []  }, 
    { "id" : 3103, "wishes" : [ 2,1,5,4,3 ], 'itv_visited' : []  },
    { "id" : 3104, "wishes" : [ 2,1,4,5,3 ], 'itv_visited' : []  }, 
    { "id" : 3105, "wishes" : [ 2,1,5,3,4 ], 'itv_visited' : []  }, 
    { "id" : 3106, "wishes" : [ 2,1,3,4,5 ], 'itv_visited' : []  }, 
    { "id" : 3107, "wishes" : [ 2,1,3,4,5 ], 'itv_visited' : []  },
    { "id" : 3108, "wishes" : [ 2,1,5,3,4 ], 'itv_visited' : []  }, 
    { "id" : 3109, "wishes" : [ 2,1,5,4,3 ], 'itv_visited' : []  }, 
    { "id" : 3110, "wishes" : [ 2,4,5,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3111, "wishes" : [ 2,4,5,1,3 ], 'itv_visited' : []  },
    { "id" : 3112, "wishes" : [ 4,2,1,3,5 ], 'itv_visited' : []  }
]
NB_SLOTS = 5

timetable, test_students = get_timetable_student_driven(NB_SLOTS, test_students, intervenants_data)
print(pd.DataFrame(timetable))
for student in test_students:
    print(student)

                    0                    1                    2   \
0  (3100, (0, (2, 0)))  (3101, (0, (2, 0)))  (3102, (0, (2, 0)))   
1  (3100, (1, (1, 0)))  (3101, (1, (1, 0)))  (3102, (1, (1, 0)))   
2  (3100, (2, (3, 0)))  (3101, (2, (3, 0)))  (3102, (2, (5, 0)))   
3  (3100, (3, (4, 0)))  (3101, (3, (5, 0)))  (3102, (3, (4, 0)))   
4  (3100, (4, (5, 0)))  (3101, (4, (4, 0)))  (3102, (4, (3, 0)))   

                    3                    4                    5   \
0  (3103, (0, (2, 0)))  (3104, (0, (2, 1)))  (3105, (0, (2, 1)))   
1  (3103, (1, (1, 0)))  (3104, (1, (1, 1)))  (3105, (1, (1, 1)))   
2  (3103, (2, (5, 0)))  (3104, (2, (4, 0)))  (3105, (2, (5, 0)))   
3  (3103, (3, (4, 0)))  (3104, (3, (5, 0)))  (3105, (3, (3, 0)))   
4  (3103, (4, (3, 0)))  (3104, (4, (3, 0)))  (3105, (4, (4, 0)))   

                    6                    7                    8   \
0  (3106, (0, (2, 1)))  (3107, (0, (2, 1)))  (3108, (0, (1, 0)))   
1  (3106, (1, (1, 1)))  (3107, (1, (1, 1)))  (

In [52]:
timetable_student_view(timetable)

defaultdict(<class 'list'>, {3100: [(0, (2, 0)), (1, (1, 0)), (2, (3, 0)), (3, (4, 0)), (4, (5, 0))], 3101: [(0, (2, 0)), (1, (1, 0)), (2, (3, 0)), (3, (5, 0)), (4, (4, 0))], 3102: [(0, (2, 0)), (1, (1, 0)), (2, (5, 0)), (3, (4, 0)), (4, (3, 0))], 3103: [(0, (2, 0)), (1, (1, 0)), (2, (5, 0)), (3, (4, 0)), (4, (3, 0))], 3104: [(0, (2, 1)), (1, (1, 1)), (2, (4, 0)), (3, (5, 0)), (4, (3, 0))], 3105: [(0, (2, 1)), (1, (1, 1)), (2, (5, 0)), (3, (3, 0)), (4, (4, 0))], 3106: [(0, (2, 1)), (1, (1, 1)), (2, (3, 0)), (3, (4, 0)), (4, (5, 0))], 3107: [(0, (2, 1)), (1, (1, 1)), (2, (3, 0)), (3, (5, 0)), (4, (4, 0))], 3108: [(0, (1, 0)), (1, (2, 0)), (2, (5, 0)), (3, (3, 0)), (4, (4, 0))], 3109: [(0, (1, 0)), (1, (2, 0)), (2, (5, 1)), (3, (3, 0)), None], 3110: [(0, (4, 0)), (1, (2, 0)), (2, (5, 1)), (3, (1, 0)), (4, (3, 0))], 3111: [(0, (4, 0)), (1, (2, 0)), (2, (5, 1)), (3, (1, 0)), (4, (3, 1))], 3112: [(0, (4, 0)), (1, (2, 1)), (2, (1, 0)), (3, (3, 0)), (4, (5, 0))]})


defaultdict(list,
            {3100: [(0, (2, 0)),
              (1, (1, 0)),
              (2, (3, 0)),
              (3, (4, 0)),
              (4, (5, 0))],
             3101: [(0, (2, 0)),
              (1, (1, 0)),
              (2, (3, 0)),
              (3, (5, 0)),
              (4, (4, 0))],
             3102: [(0, (2, 0)),
              (1, (1, 0)),
              (2, (5, 0)),
              (3, (4, 0)),
              (4, (3, 0))],
             3103: [(0, (2, 0)),
              (1, (1, 0)),
              (2, (5, 0)),
              (3, (4, 0)),
              (4, (3, 0))],
             3104: [(0, (2, 1)),
              (1, (1, 1)),
              (2, (4, 0)),
              (3, (5, 0)),
              (4, (3, 0))],
             3105: [(0, (2, 1)),
              (1, (1, 1)),
              (2, (5, 0)),
              (3, (3, 0)),
              (4, (4, 0))],
             3106: [(0, (2, 1)),
              (1, (1, 1)),
              (2, (3, 0)),
              (3, (4, 0)),
              (4

In [53]:
# test : q_students_pass1
test_intervenants_data = pd.DataFrame([
  [],
  [
    { "id" : 10, "batch_size":3 },
    { "id" : 11, "batch_size":3 }
  ],
    [
    { "id" : 20, "batch_size":3 },
    { "id" : 21, "batch_size":3 }
  ],
  [
    { "id" : 30, "batch_size":3 }  
  ]
])
test_students = [ 
    { "id" : 3100, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3101, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3102, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3103, "wishes" : [ 1,2,3 ], 'itv_visited' : []  },
    { "id" : 3104, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3105, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
]

NB_SLOTS = 3

timetable,_ = get_timetable_student_driven(NB_SLOTS, test_students, test_intervenants_data)
print(pd.DataFrame(timetable))

                     0                    1                    2  \
0  (3100, (0, (1, 0)))  (3101, (0, (1, 0)))  (3102, (0, (1, 0)))   
1  (3100, (1, (2, 0)))  (3101, (1, (2, 0)))  (3102, (1, (2, 0)))   
2  (3100, (2, (3, 0)))  (3101, (2, (3, 0)))  (3102, (2, (3, 0)))   

                     3                    4                    5  
0  (3103, (0, (1, 1)))  (3104, (0, (1, 1)))  (3105, (0, (1, 1)))  
1  (3103, (1, (2, 1)))  (3104, (1, (2, 1)))  (3105, (1, (2, 1)))  
2         (3103, None)         (3104, None)         (3105, None)  


## Queing intervenants load balance driven

In [54]:
def get_student_for_job_not_visited(students:list[Student], job:Job)->Student|None:
    for student in students:
        #print("get_student_for_job_not_visited:: processing std {} for job {}".format(student,job))
        std_wishes = student['wishes']
        for wish in std_wishes:
            if wish == job:
                j_yet_visited = [ j for (s,(j,i)) in student['itv_visited'] ]
                if not job in j_yet_visited:
                    #print("get_student_for_job_not_visited:: returning {} for job {}".format(student,job))
                    return student
    return None

In [55]:
def get_student_for_job_itv_not_visited(students:list[Student], job:Job, itv:int)->Student|None:
    for student in students:
        #print("processing std {} for job {}".format(student,job))
        std_wishes = student['wishes']
        for wish in std_wishes:
            if wish == job:
                j_i_visited = [ (j,i) for (s,(j,i)) in student['itv_visited'] ]
                if not (job,itv) in j_i_visited:
                    return student
    return None

In [56]:
def q_students_itv_driven_pass1(n_slot:int, q_intervenants:Q_Intervenants, students:list[Student])->Tuple[Q_Intervenants, list[Student], list[Student]]:
    NB_LOOP_MAX = 1000
    result_students = get_copy_students(students)

    #result_students = [ { **student, **{ 'bck_wishes': list(student['wishes']) }}  for student in result_students ]

    remain_students = result_students
    no_more_Q = False
    black_list_job=[]
    nb_loops = 0

    while((len(remain_students)>0) and (not no_more_Q)):

        #print("LOOP {}".format(nb_loops))
        assert(nb_loops < NB_LOOP_MAX)
        nb_loops += 1

        tmp_students = remain_students
        remain_students = []

        # get next Q avail to call
        next_q_ids = [ q[0] for q in get_next_q_to_load_sorted(q_intervenants) ]
        #print("next_q_ids : {}".format(next_q_ids))

        l_q_job_itv = [ get_job_itv_from_itv_id(q_intervenants, n_q_id) for n_q_id in next_q_ids  ]
        #print("l_q_job_itv : {}".format(l_q_job_itv))

        next_q_jobs = [ (j,i) for (j,i) in l_q_job_itv if not j in black_list_job ]


        #print("next_q_jobs : {}".format(next_q_jobs))
        
        if (len(next_q_jobs)==0):
            remain_students = tmp_students
            #print('no more Q')
            no_more_Q = True
        else:

            job, itv = next_q_jobs[0]
            
            # call student avec job correspondant    
            std = get_student_for_job_not_visited(tmp_students,job)
            if std != None:

                r_student = [ s for s in result_students if s['id'] == std['id'] ][0]
                r_student['itv_visited'].append((n_slot, (job, itv)))

                q_intervenants = push_student_in_q(q_intervenants, job, itv, std['id'])

                # suppress student from remain students
                remain_students = [ s for s in tmp_students if s['id'] != std['id']]
                #print("student {} pushed in {}".format(std['id'], (job, itv)))
            else:
                black_list_job.append(job)
                #print("black list jobs : {}".format(black_list_job))
                remain_students = tmp_students

        #print("new remain_students : {}".format(remain_students))

    #next_q_ids = [ q[0] for q in get_next_q_to_load_sorted(q_intervenants) ]
    #print("next_q_ids : {}".format(next_q_ids))

    return q_intervenants, result_students, remain_students

In [57]:
def q_students_itv_driven_pass2(n_slot:int, q_intervenants:Q_Intervenants, students:list[Student], remain_students:list[Student])->Tuple[Q_Intervenants, list[Student]]:
    result_students = get_copy_students(students)

    for student in remain_students:
        # get next Q avail to call
        next_q_ids = [ q[0] for q in get_next_q_to_load_sorted(q_intervenants) ]
        #print("next_q_ids : {}".format(next_q_ids))

        l_q_job_itv = [ get_job_itv_from_itv_id(q_intervenants, n_q_id) for n_q_id in next_q_ids  ]
        #print("l_q_job_itv : {}".format(l_q_job_itv))

        job_yet_visited = { j for (s,(j,_)) in student['itv_visited'] }
        job_itv_yet_visited = { (j,i) for (s,(j,i)) in student['itv_visited'] }

        # liste des (job, itv) pour lesquels job n'a pas encore été visité par l'élève
        next_q_job_itv_1 = [ (j,i) for (j,i) in l_q_job_itv if not j in job_yet_visited ]

        # liste des (job, itv) pour lesquels (job,itv) n'a pas encore été visité
        next_q_job_itv_2 = [ (j,i) for (j,i) in l_q_job_itv if not (j,i) in job_itv_yet_visited ]

        job, itv = None, None

        if (len(next_q_job_itv_1)==0):
            if (len(next_q_job_itv_2)==0):
                print("/!\ free student")
            else:
                # solution par défaut: on refait visiter un même job mais pas le même intervenant
                job, itv = next_q_job_itv_2[0]
        else:
            # solution préférentielle : on fait visiter un nouveau job
            job, itv = next_q_job_itv_1[0]

        if job != None:
            r_student = [ s for s in result_students if s['id'] == student['id'] ][0]
            r_student['itv_visited'].append((n_slot, (job, itv)))
            q_intervenants = push_student_in_q(q_intervenants, job, itv, student['id'])

    return q_intervenants, result_students



In [58]:
# test : q_students_itv_driven
test_intervenants_data = pd.DataFrame([
  [],
  [ { "id" : 10, "batch_size":3 }, { "id" : 11, "batch_size":2 } ],
  [ { "id" : 20, "batch_size":3 }, { "id" : 21, "batch_size":2 } ],
  [ { "id" : 30, "batch_size":2 } ],
  [ { "id" : 40, "batch_size":1 } ],
  [ { "id" : 50, "batch_size":1 } ]
])

test_students = [ 
    { "id" : 3100, "wishes" : [ 2,1,3 ], 'itv_visited' : [] }, 
    { "id" : 3101, "wishes" : [ 2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3102, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3103, "wishes" : [ 2,1,4 ], 'itv_visited' : []  },
    { "id" : 3104, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3105, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3106, "wishes" : [ 2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3107, "wishes" : [ 1,2,3 ], 'itv_visited' : []  },
    { "id" : 3108, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3109, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3110, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3111, "wishes" : [ 1,2,4 ], 'itv_visited' : []  },
    { "id" : 3112, "wishes" : [ 1,3,4 ], 'itv_visited' : []  }
]

test_q_intervenants = init_q_intervenants(test_intervenants_data)
test_q_intervenants, test_students, remain_students = q_students_itv_driven_pass1(0, test_q_intervenants, test_students)

In [59]:
test_q_intervenants, test_students = q_students_itv_driven_pass2(0, test_q_intervenants, test_students, remain_students)
(test_q_intervenants, test_students)

([[],
  [{'id': 10, 'batch_size': 3, 'Q': [3100, 3105, 3111]},
   {'id': 11, 'batch_size': 2, 'Q': [3101, 3108]}],
  [{'id': 20, 'batch_size': 3, 'Q': [3103, 3107]},
   {'id': 21, 'batch_size': 2, 'Q': [3102, 3109]}],
  [{'id': 30, 'batch_size': 2, 'Q': [3106, 3110]}],
  [{'id': 40, 'batch_size': 1, 'Q': [3104]}],
  [{'id': 50, 'batch_size': 1, 'Q': [3112]}]],
 [{'id': 3100, 'wishes': [2, 1, 3], 'itv_visited': [(0, (1, 0))]},
  {'id': 3101, 'wishes': [2, 1, 3], 'itv_visited': [(0, (1, 1))]},
  {'id': 3102, 'wishes': [2, 1, 4], 'itv_visited': [(0, (2, 1))]},
  {'id': 3103, 'wishes': [2, 1, 4], 'itv_visited': [(0, (2, 0))]},
  {'id': 3104, 'wishes': [2, 1, 4], 'itv_visited': [(0, (4, 0))]},
  {'id': 3105, 'wishes': [2, 1, 4], 'itv_visited': [(0, (1, 0))]},
  {'id': 3106, 'wishes': [2, 1, 3], 'itv_visited': [(0, (3, 0))]},
  {'id': 3107, 'wishes': [1, 2, 3], 'itv_visited': [(0, (2, 0))]},
  {'id': 3108, 'wishes': [1, 2, 3], 'itv_visited': [(0, (1, 1))]},
  {'id': 3109, 'wishes': [1, 2, 3]

In [60]:
def get_timetable_itv_driven(nb_slots:int, students:list[Student], intervenants_data:pd.DataFrame)->Tuple[Timetable, list[Student]]:
    timetable = []
    for n_slot in range(nb_slots):
        #print("MAKING SLOT {}".format(n_slot))

        q_intervenants = init_q_intervenants(intervenants_data)
        q_intervenants, students, remain_students = q_students_itv_driven_pass1(n_slot, q_intervenants, students)
        
        #print("after pass 1 - students {}".format(students))
        
        q_intervenants, students = q_students_itv_driven_pass2(n_slot, q_intervenants, students, remain_students)
        timetable.append(q_intervenants)
    return timetable, students

## Timetable for intervenants

In [61]:
def get_id_from_j_i(l_id_j_i:list[TimeslotStudentVisit], j_i:Tuple[Job,Itv])->Id:
    """
    [ (id, (job, itv)),... ]  
    -> id correspondant à (j,i) fourni en argument
    """
    j, i = j_i
    l_res = [ e_id for (e_id, (e_j,e_i)) in l_id_j_i if (e_j==j) and (e_i==i)]
    if (len(l_res) == 0):
        return None
    else :
        return l_res[0]    

In [62]:
def get_timetable_intervenants(timetable:Timetable, q_intervenants:Q_Intervenants)->list[dict[int,int]]:

    
    #l_itv_visits = [ (slot, itv_visit) for slot in timetable for  (_, (slot, itv_visit)) in slot ]
    l_itv_visits = [ (s, itv_visit) for (s,(itv_visit)) in [ e for slot in timetable for  (_, e) in slot if e != None] ]
    #print(l_itv_visits)

    time_slots = list({ s for (_,(s,_)) in [ (id, itv_visit) for slot in timetable for id, itv_visit in slot if itv_visit != None ] })
    print("nb_slots : {}".format(time_slots))

    l_itv_id_job_itv = build_id_to_job_itv(q_intervenants)
    l_ids_itv = [ id for (id,_) in l_itv_id_job_itv]

    #print(l_itv_id_job_itv)

    #def get_id_from_j_i(l_id_j_i, j_i):
    #    j, i = j_i
    #    l_res = [ e_id for (e_id, (e_j,e_i)) in l_id_j_i if (e_j==j) and (e_i==i)]
    #    if (len(l_res) == 0):
    #        return None
    #    else :
    #        return l_res[0]

    l_result = []
    for n_slot in time_slots:

        l_slot_itv_visits = [ (slot,itv_visit) for (slot, itv_visit) in l_itv_visits if slot == n_slot ]
        itv_entries = defaultdict(int)
        for id in l_ids_itv:
            itv_entries[id]=0

        for slot_itv_visit in l_slot_itv_visits :
            slot, (j,i) = slot_itv_visit
            id = get_id_from_j_i(l_itv_id_job_itv, (j,i))
            if id != None:
                itv_entries[id] += 1
                #print("entry : id {} value {}".format(id,itv_entries[id]))

        l_result.append(dict(itv_entries))

    return l_result

## Evaluate

In [63]:
def evaluate_1(timetable:Timetable, students:list[Student], q_intervenants:Q_Intervenants)->None:
    tmp_students = get_copy_students(students)
    overall_student_satisfaction = 0
    overall_ratio_itv_visited = 0
    
    for student in tmp_students:
        l_w_fullfilled = [ w for w in student['wishes'] if w == -1 ]
        student_satisfaction = round(len(l_w_fullfilled) / len(student['wishes']),3)
        #print("satisf. {}".format(student_satisfaction))
        overall_student_satisfaction += student_satisfaction

        ratio_itv_visited = round(len(student['itv_visited']) / len(student['wishes']),3)
        overall_ratio_itv_visited += ratio_itv_visited

    overall_ratio_itv_visited = round(overall_ratio_itv_visited/len(students),3)
    overall_student_satisfaction = round(overall_student_satisfaction / len(tmp_students),3)

    print("overall_student_satisfaction : {}".format(overall_student_satisfaction))
    print("overall ratio visited {}".format(overall_ratio_itv_visited))

    timetable_itv = get_timetable_intervenants(timetable, q_intervenants)
    #print("timetable_itv {}".format(timetable_itv))

    l_n_visits = np.array([ n_visits for slot in timetable_itv for n_visits in slot.values() ])
    nb_itv_no_visit = np.argwhere(l_n_visits == 0).size
    print("nb_no_visits_itv : {}".format(nb_itv_no_visit))


In [64]:
def evaluate_2(timetable:Timetable, students:list[Student], q_intervenants:Q_Intervenants)->None:
    tmp_students = get_copy_students(students)
    overall_student_satisfaction = 0
    overall_ratio_itv_visited = 0

    for student in tmp_students:
        l_job_visited = { j for (s,(j,_)) in student['itv_visited'] }
        #print("l_job_visited {}".format(l_job_visited))
        l_w_fullfilled = [ w for w in student['wishes'] if w in l_job_visited ]
        student_satisfaction = round(len(l_w_fullfilled) / len(student['wishes']),3)
        #print("satisf. {}".format(student_satisfaction))
        overall_student_satisfaction += student_satisfaction

        ratio_itv_visited = round(len(student['itv_visited']) / len(student['wishes']),3)
        overall_ratio_itv_visited += ratio_itv_visited

    overall_ratio_itv_visited = round(overall_ratio_itv_visited/len(students),3)

    overall_student_satisfaction = round(overall_student_satisfaction / len(tmp_students),3)
    print("overall_student_satisfaction : {}".format(overall_student_satisfaction))
    print("overall ratio visited {}".format(overall_ratio_itv_visited))

    timetable_itv = get_timetable_intervenants(timetable, q_intervenants)
    #print("timetable_itv {}".format(timetable_itv))

    l_n_visits = np.array([ n_visits for slot in timetable_itv for n_visits in slot.values() ])
    nb_itv_no_visit = np.argwhere(l_n_visits == 0).size
    print("nb_no_visits_itv : {}".format(nb_itv_no_visit))

    

    

## Tests

### Student driven

In [65]:
test_intervenants_data = pd.DataFrame([
  [],
  [ { "id" : 10, "batch_size":3 }, { "id" : 11, "batch_size":2 } ],
  [ { "id" : 20, "batch_size":3 }, { "id" : 21, "batch_size":2 } ],
  [ { "id" : 30, "batch_size":2 } ],
  [ { "id" : 40, "batch_size":1 } ],
  [ { "id" : 50, "batch_size":1 } ]
])

test_students = [ 
    { "id" : 3100, "wishes" : [ 2,1,3 ], 'itv_visited' : [] }, 
    { "id" : 3101, "wishes" : [ 2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3102, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3103, "wishes" : [ 2,1,4 ], 'itv_visited' : []  },
    { "id" : 3104, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3105, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3106, "wishes" : [ 2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3107, "wishes" : [ 1,2,3 ], 'itv_visited' : []  },
    { "id" : 3108, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3109, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3110, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3111, "wishes" : [ 1,2,4 ], 'itv_visited' : []  },
    { "id" : 3112, "wishes" : [ 1,3,4 ], 'itv_visited' : []  }
]

NB_SLOTS=3
timetable, test_students = get_timetable_student_driven(NB_SLOTS, test_students, test_intervenants_data)
pd.DataFrame(timetable)


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,"(3100, (0, (2, 0)))","(3101, (0, (2, 0)))","(3102, (0, (2, 0)))","(3103, (0, (2, 1)))","(3104, (0, (2, 1)))","(3105, (0, (4, 0)))","(3106, (0, (3, 0)))","(3107, (0, (1, 0)))","(3108, (0, (1, 0)))","(3109, (0, (1, 0)))","(3110, (0, (1, 1)))","(3111, (0, (1, 1)))","(3112, (0, (3, 0)))"
1,"(3100, (1, (1, 0)))","(3101, (1, (1, 0)))","(3102, (1, (1, 0)))","(3103, (1, (1, 1)))","(3104, (1, (1, 1)))","(3105, (1, (2, 0)))","(3106, (1, (2, 0)))","(3107, (1, (2, 0)))","(3108, (1, (2, 1)))","(3109, (1, (2, 1)))","(3110, (1, (3, 0)))","(3111, (1, (4, 0)))","(3112, (2, (1, 0)))"
2,"(3100, (2, (3, 0)))","(3101, (2, (3, 0)))","(3102, (2, (4, 0)))","(3103, None)","(3104, None)","(3105, (2, (1, 0)))","(3106, (2, (1, 0)))","(3107, None)","(3108, None)","(3109, None)","(3110, (2, (2, 0)))","(3111, (2, (2, 0)))","(3112, None)"


In [66]:
test_q_intervenants = init_q_intervenants(test_intervenants_data)

print(evaluate_1(timetable, test_students, test_q_intervenants))

overall_student_satisfaction : 0.846
overall ratio visited 0.846
nb_slots : [0, 1, 2]
nb_no_visits_itv : 5
None


In [67]:
timetable

[[(3100, (0, (2, 0))),
  (3101, (0, (2, 0))),
  (3102, (0, (2, 0))),
  (3103, (0, (2, 1))),
  (3104, (0, (2, 1))),
  (3105, (0, (4, 0))),
  (3106, (0, (3, 0))),
  (3107, (0, (1, 0))),
  (3108, (0, (1, 0))),
  (3109, (0, (1, 0))),
  (3110, (0, (1, 1))),
  (3111, (0, (1, 1))),
  (3112, (0, (3, 0)))],
 [(3100, (1, (1, 0))),
  (3101, (1, (1, 0))),
  (3102, (1, (1, 0))),
  (3103, (1, (1, 1))),
  (3104, (1, (1, 1))),
  (3105, (1, (2, 0))),
  (3106, (1, (2, 0))),
  (3107, (1, (2, 0))),
  (3108, (1, (2, 1))),
  (3109, (1, (2, 1))),
  (3110, (1, (3, 0))),
  (3111, (1, (4, 0))),
  (3112, (2, (1, 0)))],
 [(3100, (2, (3, 0))),
  (3101, (2, (3, 0))),
  (3102, (2, (4, 0))),
  (3103, None),
  (3104, None),
  (3105, (2, (1, 0))),
  (3106, (2, (1, 0))),
  (3107, None),
  (3108, None),
  (3109, None),
  (3110, (2, (2, 0))),
  (3111, (2, (2, 0))),
  (3112, None)]]

In [68]:
test_q_intervenants = init_q_intervenants(test_intervenants_data)

pd.DataFrame(get_timetable_intervenants(timetable, test_q_intervenants))
#get_timetable_intervenants(timetable, test_intervenants_data)

nb_slots : [0, 1, 2]


,10,11,20,21,30,40,50
0,3,2,3,2,2,1,0
1,3,2,3,2,1,1,0
2,3,0,2,0,2,1,0


In [69]:
timetable_student_view(timetable)

defaultdict(<class 'list'>, {3100: [(0, (2, 0)), (1, (1, 0)), (2, (3, 0))], 3101: [(0, (2, 0)), (1, (1, 0)), (2, (3, 0))], 3102: [(0, (2, 0)), (1, (1, 0)), (2, (4, 0))], 3103: [(0, (2, 1)), (1, (1, 1)), None], 3104: [(0, (2, 1)), (1, (1, 1)), None], 3105: [(0, (4, 0)), (1, (2, 0)), (2, (1, 0))], 3106: [(0, (3, 0)), (1, (2, 0)), (2, (1, 0))], 3107: [(0, (1, 0)), (1, (2, 0)), None], 3108: [(0, (1, 0)), (1, (2, 1)), None], 3109: [(0, (1, 0)), (1, (2, 1)), None], 3110: [(0, (1, 1)), (1, (3, 0)), (2, (2, 0))], 3111: [(0, (1, 1)), (1, (4, 0)), (2, (2, 0))], 3112: [(0, (3, 0)), (2, (1, 0)), None]})


defaultdict(list,
            {3100: [(0, (2, 0)), (1, (1, 0)), (2, (3, 0))],
             3101: [(0, (2, 0)), (1, (1, 0)), (2, (3, 0))],
             3102: [(0, (2, 0)), (1, (1, 0)), (2, (4, 0))],
             3103: [(0, (2, 1)), (1, (1, 1)), None],
             3104: [(0, (2, 1)), (1, (1, 1)), None],
             3105: [(0, (4, 0)), (1, (2, 0)), (2, (1, 0))],
             3106: [(0, (3, 0)), (1, (2, 0)), (2, (1, 0))],
             3107: [(0, (1, 0)), (1, (2, 0)), None],
             3108: [(0, (1, 0)), (1, (2, 1)), None],
             3109: [(0, (1, 0)), (1, (2, 1)), None],
             3110: [(0, (1, 1)), (1, (3, 0)), (2, (2, 0))],
             3111: [(0, (1, 1)), (1, (4, 0)), (2, (2, 0))],
             3112: [(0, (3, 0)), (2, (1, 0)), None]})

In [70]:
pd.DataFrame(timetable_student_view(timetable)).T

defaultdict(<class 'list'>, {3100: [(0, (2, 0)), (1, (1, 0)), (2, (3, 0))], 3101: [(0, (2, 0)), (1, (1, 0)), (2, (3, 0))], 3102: [(0, (2, 0)), (1, (1, 0)), (2, (4, 0))], 3103: [(0, (2, 1)), (1, (1, 1)), None], 3104: [(0, (2, 1)), (1, (1, 1)), None], 3105: [(0, (4, 0)), (1, (2, 0)), (2, (1, 0))], 3106: [(0, (3, 0)), (1, (2, 0)), (2, (1, 0))], 3107: [(0, (1, 0)), (1, (2, 0)), None], 3108: [(0, (1, 0)), (1, (2, 1)), None], 3109: [(0, (1, 0)), (1, (2, 1)), None], 3110: [(0, (1, 1)), (1, (3, 0)), (2, (2, 0))], 3111: [(0, (1, 1)), (1, (4, 0)), (2, (2, 0))], 3112: [(0, (3, 0)), (2, (1, 0)), None]})


,0,1,2
3100,"(0, (2, 0))","(1, (1, 0))","(2, (3, 0))"
3101,"(0, (2, 0))","(1, (1, 0))","(2, (3, 0))"
3102,"(0, (2, 0))","(1, (1, 0))","(2, (4, 0))"
3103,"(0, (2, 1))","(1, (1, 1))",None
3104,"(0, (2, 1))","(1, (1, 1))",None
3105,"(0, (4, 0))","(1, (2, 0))","(2, (1, 0))"
3106,"(0, (3, 0))","(1, (2, 0))","(2, (1, 0))"
3107,"(0, (1, 0))","(1, (2, 0))",None
3108,"(0, (1, 0))","(1, (2, 1))",None
3109,"(0, (1, 0))","(1, (2, 1))",None


### Intervenant driven

In [71]:
# test : get_timetable_itv_drivent

test_intervenants_data = pd.DataFrame([
  [],
  [ { "id" : 10, "batch_size":3 }, { "id" : 11, "batch_size":2 } ],
  [ { "id" : 20, "batch_size":3 }, { "id" : 21, "batch_size":2 } ],
  [ { "id" : 30, "batch_size":2 } ],
  [ { "id" : 40, "batch_size":1 } ],
  [ { "id" : 50, "batch_size":1 } ]
])

test_students = [ 
    { "id" : 3100, "wishes" : [ 2,1,3 ], 'itv_visited' : [] }, 
    { "id" : 3101, "wishes" : [ 2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3102, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3103, "wishes" : [ 2,1,4 ], 'itv_visited' : []  },
    { "id" : 3104, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3105, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3106, "wishes" : [ 2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3107, "wishes" : [ 1,2,3 ], 'itv_visited' : []  },
    { "id" : 3108, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3109, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3110, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3111, "wishes" : [ 1,2,4 ], 'itv_visited' : []  },
    { "id" : 3112, "wishes" : [ 1,3,4 ], 'itv_visited' : []  }
]


NB_SLOTS = 3
_, test_students = get_timetable_itv_driven(NB_SLOTS, test_students, test_intervenants_data)
timetable = convert_to_student_timetable(NB_SLOTS, test_students)


In [72]:
timetable

[[(3100, (0, (1, 0))),
  (3101, (0, (1, 1))),
  (3102, (0, (2, 1))),
  (3103, (0, (2, 0))),
  (3104, (0, (4, 0))),
  (3105, (0, (1, 0))),
  (3106, (0, (3, 0))),
  (3107, (0, (2, 0))),
  (3108, (0, (1, 1))),
  (3109, (0, (2, 1))),
  (3110, (0, (3, 0))),
  (3111, (0, (1, 0))),
  (3112, (0, (5, 0)))],
 [(3100, (1, (2, 1))),
  (3101, (1, (2, 0))),
  (3102, (1, (1, 0))),
  (3103, (1, (1, 1))),
  (3104, (1, (1, 0))),
  (3105, (1, (4, 0))),
  (3106, (1, (2, 0))),
  (3107, (1, (3, 0))),
  (3108, (1, (2, 1))),
  (3109, (1, (1, 1))),
  (3110, (1, (1, 0))),
  (3111, (1, (2, 0))),
  (3112, (1, (3, 0)))],
 [(3100, (2, (3, 0))),
  (3101, (2, (3, 0))),
  (3102, (2, (4, 0))),
  (3103, (2, (5, 0))),
  (3104, (2, (2, 1))),
  (3105, (2, (2, 0))),
  (3106, (2, (1, 0))),
  (3107, (2, (1, 1))),
  (3108, (2, (1, 0))),
  (3109, (2, (2, 0))),
  (3110, (2, (2, 0))),
  (3111, (2, (1, 1))),
  (3112, (2, (1, 0)))]]

In [73]:
#pd.DataFrame(get_timetable_intervenants(timetable, test_intervenants_data))
test_q_intervenants = init_q_intervenants(test_intervenants_data)
get_timetable_intervenants(timetable, test_q_intervenants)

nb_slots : [0, 1, 2]


[{10: 3, 11: 2, 20: 2, 21: 2, 30: 2, 40: 1, 50: 1},
 {10: 3, 11: 2, 20: 3, 21: 2, 30: 2, 40: 1, 50: 0},
 {10: 3, 11: 2, 20: 3, 21: 1, 30: 2, 40: 1, 50: 1}]

In [74]:
evaluate_2(timetable, test_students, test_q_intervenants)

overall_student_satisfaction : 0.872
overall ratio visited 1.0
nb_slots : [0, 1, 2]
nb_no_visits_itv : 1


In [75]:
timetable_student_view(timetable)

defaultdict(<class 'list'>, {3100: [(0, (1, 0)), (1, (2, 1)), (2, (3, 0))], 3101: [(0, (1, 1)), (1, (2, 0)), (2, (3, 0))], 3102: [(0, (2, 1)), (1, (1, 0)), (2, (4, 0))], 3103: [(0, (2, 0)), (1, (1, 1)), (2, (5, 0))], 3104: [(0, (4, 0)), (1, (1, 0)), (2, (2, 1))], 3105: [(0, (1, 0)), (1, (4, 0)), (2, (2, 0))], 3106: [(0, (3, 0)), (1, (2, 0)), (2, (1, 0))], 3107: [(0, (2, 0)), (1, (3, 0)), (2, (1, 1))], 3108: [(0, (1, 1)), (1, (2, 1)), (2, (1, 0))], 3109: [(0, (2, 1)), (1, (1, 1)), (2, (2, 0))], 3110: [(0, (3, 0)), (1, (1, 0)), (2, (2, 0))], 3111: [(0, (1, 0)), (1, (2, 0)), (2, (1, 1))], 3112: [(0, (5, 0)), (1, (3, 0)), (2, (1, 0))]})


defaultdict(list,
            {3100: [(0, (1, 0)), (1, (2, 1)), (2, (3, 0))],
             3101: [(0, (1, 1)), (1, (2, 0)), (2, (3, 0))],
             3102: [(0, (2, 1)), (1, (1, 0)), (2, (4, 0))],
             3103: [(0, (2, 0)), (1, (1, 1)), (2, (5, 0))],
             3104: [(0, (4, 0)), (1, (1, 0)), (2, (2, 1))],
             3105: [(0, (1, 0)), (1, (4, 0)), (2, (2, 0))],
             3106: [(0, (3, 0)), (1, (2, 0)), (2, (1, 0))],
             3107: [(0, (2, 0)), (1, (3, 0)), (2, (1, 1))],
             3108: [(0, (1, 1)), (1, (2, 1)), (2, (1, 0))],
             3109: [(0, (2, 1)), (1, (1, 1)), (2, (2, 0))],
             3110: [(0, (3, 0)), (1, (1, 0)), (2, (2, 0))],
             3111: [(0, (1, 0)), (1, (2, 0)), (2, (1, 1))],
             3112: [(0, (5, 0)), (1, (3, 0)), (2, (1, 0))]})

## Tests données réelles

### Student driven

In [111]:
test_intervenants_data = pd.read_json('intervenants_data.json')

# importation d'un échantillon de données pour les tests unitaires
students_2025 = pd.read_json('data-2025.json')
result = students_2025.to_json(orient="index")
parsed = loads(result)
test_students_2025 = list(parsed.values())

In [112]:

NB_SLOTS = 6
timetable, test_students_2025 = get_timetable_student_driven(NB_SLOTS, test_students_2025, test_intervenants_data)
#timetable


In [113]:

timetable = convert_to_student_timetable(NB_SLOTS, test_students_2025)
timetable

[[(3100, (0, (8, 0))),
  (3101, (0, (3, 0))),
  (3102, (0, (9, 0))),
  (3103, (0, (17, 0))),
  (3104, (0, (10, 0))),
  (3105, (0, (6, 0))),
  (3106, (0, (9, 0))),
  (3107, (0, (9, 0))),
  (3108, (0, (11, 0))),
  (3109, (0, (1, 0))),
  (3110, (0, (1, 0))),
  (3111, (0, (10, 0))),
  (3112, (0, (8, 0))),
  (3113, (0, (7, 0))),
  (3114, (0, (14, 0))),
  (3115, (0, (4, 0))),
  (3116, (0, (8, 0))),
  (3117, (0, (5, 0))),
  (3118, (0, (13, 0))),
  (3119, (0, (7, 0))),
  (3120, (0, (7, 0))),
  (3121, (0, (8, 0))),
  (3122, (0, (18, 0))),
  (3123, (0, (9, 0))),
  (3124, (0, (11, 0))),
  (3125, (0, (11, 0))),
  (3126, (0, (16, 0))),
  (3127, (0, (16, 0))),
  (3200, (0, (18, 0))),
  (3201, (0, (1, 0))),
  (3202, (0, (10, 0))),
  (3203, (0, (1, 0))),
  (3204, (0, (14, 0))),
  (3205, (0, (10, 0))),
  (3206, (0, (1, 1))),
  (3207, (0, (20, 0))),
  (3208, (0, (1, 1))),
  (3209, (0, (8, 1))),
  (3210, (0, (4, 0))),
  (3211, (0, (18, 0))),
  (3212, (0, (10, 1))),
  (3213, (0, (12, 0))),
  (3214, (0, (9

In [114]:
q_intervenants = init_q_intervenants(test_intervenants_data)

pd.set_option("display.max_columns", 65)
pd.set_option("display.max_rows", 6)
pd.DataFrame(get_timetable_intervenants(timetable, q_intervenants))
#get_timetable_intervenants(timetable, test_intervenants_data)

nb_slots : [0, 1, 2, 3, 4, 5]


,10,11,12,20,21,30,31,40,50,51,52,53,54,55,60,61,62,63,70,80,81,82,83,90,91,92,93,94,95,100,101,102,110,111,112,113,114,120,130,131,140,141,142,150,151,160,161,162,163,170,171,172,180,181,182,190,191,192,200,210,211
0,4,4,4,4,4,4,4,4,4,4,4,4,4,2,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,0,4,4,4,4,4,4,4,2,4,4,3,0,4,2,0,4,4,4,4,3,0,4,4,4
1,4,4,4,4,4,4,4,4,4,4,3,0,0,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,0,4,4,4,4,4,4,4,0,4,4,4,2,4,0,0,4,4,4,4,1,0,4,4,3
2,4,4,4,4,4,4,4,4,4,4,3,0,0,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,0,0,4,4,1,4,4,4,4,1,4,2,0,0,4,0,0,4,4,4,4,4,3,4,4,0
3,4,4,4,4,4,4,4,4,4,3,0,0,0,0,4,4,4,4,4,4,4,4,4,4,4,4,4,2,0,4,4,4,3,0,0,0,0,4,4,3,4,4,4,4,2,4,4,1,0,4,0,0,4,4,3,4,4,0,4,4,1
4,4,4,4,4,4,4,1,4,4,0,0,0,0,0,4,4,4,4,4,4,4,4,4,4,4,4,4,0,0,4,4,4,4,4,0,0,0,4,2,0,4,4,4,2,0,4,4,0,0,3,0,0,4,0,0,2,0,0,4,1,0
5,4,4,4,4,4,0,0,4,0,0,0,0,0,0,4,4,4,4,4,4,4,1,0,0,0,0,0,0,0,4,4,4,0,0,0,0,0,4,0,0,4,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,4,0,0


In [119]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 240)
timetable_students = pd.DataFrame(timetable_student_view(timetable)).T
timetable_students
timetable_students.to_csv('output_timetable_students_std_driven.csv', sep=';', header=False)

defaultdict(<class 'list'>, {3100: ['8-0', '3-0', '7-0', '6-0', '9-0', '-1-0'], 3101: ['3-0', '7-0', '6-0', '19-0', '11-0', '-1-0'], 3102: ['9-0', '10-0', '16-0', '6-0', '11-0', '-1-0'], 3103: ['17-0', '16-0', '7-0', '8-0', '9-0', '-1-0'], 3104: ['10-0', '11-0', '12-0', '16-0', '2-0', '-1-0'], 3105: ['6-0', '10-0', '11-0', '12-0', '3-0', '-1-0'], 3106: ['9-0', '10-0', '8-0', '20-0', '16-0', '-1-0'], 3107: ['9-0', '20-0', '6-0', '16-0', '11-0', '-1-0'], 3108: ['11-0', '10-0', '13-0', '3-0', '7-0', '-1-0'], 3109: ['1-0', '2-0', '3-0', '4-0', '5-0', '-1-0'], 3110: ['1-0', '18-0', '4-0', '2-0', '9-0', '-1-0'], 3111: ['10-0', '5-0', '18-0', '8-0', '9-0', '-1-0'], 3112: ['8-0', '6-0', '4-0', '5-0', '9-1', '-1-0'], 3113: ['7-0', '20-0', '6-0', '3-0', '8-0', '-1-0'], 3114: ['14-0', '9-0', '2-0', '21-0', '8-0', '-1-0'], 3115: ['4-0', '5-0', '16-0', '9-0', '20-0', '-1-0'], 3116: ['8-0', '10-1', '6-0', '9-0', '16-0', '-1-0'], 3117: ['5-0', '6-0', '20-0', '9-0', '14-0', '-1-0'], 3118: ['13-0', '21

### Intervenants drivent

In [ ]:
test_intervenants_data = pd.read_json('intervenants_data.json')

# importation d'un échantillon de données pour les tests unitaires
students_2025 = pd.read_json('data-2025.json')
result = students_2025.to_json(orient="index")
parsed = loads(result)
test_students_2025 = list(parsed.values())

In [ ]:

NB_SLOTS = 6
timetable, test_students_2025 = get_timetable_itv_driven(NB_SLOTS, test_students_2025, test_intervenants_data)
#timetable


In [ ]:

timetable = convert_to_student_timetable(NB_SLOTS, test_students_2025)
#timetable

In [ ]:
q_intervenants = init_q_intervenants(test_intervenants_data)

pd.set_option("display.max_columns", 65)
pd.set_option("display.max_rows", 6)
pd.DataFrame(get_timetable_intervenants(timetable, q_intervenants))
#get_timetable_intervenants(timetable, test_intervenants_data)

nb_slots : [0, 1, 2, 3, 4, 5]


,10,11,12,20,21,30,31,40,50,51,52,53,54,55,60,61,62,63,70,80,81,82,83,90,91,92,93,94,95,100,101,102,110,111,112,113,114,120,130,131,140,141,142,150,151,160,161,162,163,170,171,172,180,181,182,190,191,192,200,210,211
0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,3,3,4,4,3,4,4,4,4,3,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
1,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,3,3,4,3,3,4,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
2,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,3,3,3,3,4,4,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,3,3,3,4,4,4,3,4,4,4,4,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4
4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,3,3,3,3,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
5,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,3,3,4,3,4,4,4,4,4,4,4,4,4,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4


In [95]:
evaluate_2(timetable, test_students_2025, q_intervenants)

overall_student_satisfaction : 0.862
overall ratio visited 1.2
nb_slots : [0, 1, 2, 3, 4, 5]
nb_no_visits_itv : 0


In [96]:
timetable_student_view(timetable)

defaultdict(<class 'list'>, {3100: ['3-0', '6-1', '9-0', '7-0', '8-0', '5-5'], 3101: ['3-1', '6-0', '7-0', '19-2', '11-4', '5-1'], 3102: ['6-1', '10-2', '9-1', '16-1', '11-3', '3-1'], 3103: ['9-0', '8-0', '16-2', '17-2', '7-0', '3-0'], 3104: ['2-0', '10-0', '16-3', '12-0', '11-2', '5-4'], 3105: ['6-0', '3-0', '10-2', '11-4', '12-0', '5-2'], 3106: ['10-2', '9-0', '8-0', '20-0', '16-0', '5-3'], 3107: ['9-1', '6-3', '16-1', '11-1', '20-0', '5-0'], 3108: ['10-0', '3-1', '13-1', '11-2', '7-0', '9-5'], 3109: ['1-0', '2-0', '4-0', '3-0', '5-3', '9-1'], 3110: ['1-1', '4-0', '2-0', '9-2', '18-2', '17-2'], 3111: ['5-3', '9-1', '10-0', '8-0', '18-1', '15-1'], 3112: ['4-0', '5-3', '6-1', '9-3', '8-2', '18-0'], 3113: ['8-0', '7-0', '3-0', '6-1', '20-0', '9-4'], 3114: ['2-1', '9-2', '8-1', '21-0', '14-2', '17-1'], 3115: ['5-4', '9-4', '16-0', '4-0', '20-0', '18-2'], 3116: ['9-2', '10-1', '6-0', '8-3', '16-1', '19-0'], 3117: ['5-5', '9-3', '6-3', '14-1', '20-0', '18-1'], 3118: ['8-1', '6-2', '20-0', 

defaultdict(list,
            {3100: ['3-0', '6-1', '9-0', '7-0', '8-0', '5-5'],
             3101: ['3-1', '6-0', '7-0', '19-2', '11-4', '5-1'],
             3102: ['6-1', '10-2', '9-1', '16-1', '11-3', '3-1'],
             3103: ['9-0', '8-0', '16-2', '17-2', '7-0', '3-0'],
             3104: ['2-0', '10-0', '16-3', '12-0', '11-2', '5-4'],
             3105: ['6-0', '3-0', '10-2', '11-4', '12-0', '5-2'],
             3106: ['10-2', '9-0', '8-0', '20-0', '16-0', '5-3'],
             3107: ['9-1', '6-3', '16-1', '11-1', '20-0', '5-0'],
             3108: ['10-0', '3-1', '13-1', '11-2', '7-0', '9-5'],
             3109: ['1-0', '2-0', '4-0', '3-0', '5-3', '9-1'],
             3110: ['1-1', '4-0', '2-0', '9-2', '18-2', '17-2'],
             3111: ['5-3', '9-1', '10-0', '8-0', '18-1', '15-1'],
             3112: ['4-0', '5-3', '6-1', '9-3', '8-2', '18-0'],
             3113: ['8-0', '7-0', '3-0', '6-1', '20-0', '9-4'],
             3114: ['2-1', '9-2', '8-1', '21-0', '14-2', '17-1'],
    

In [98]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 240)
timetable_students = pd.DataFrame(timetable_student_view(timetable)).T
timetable_students

defaultdict(<class 'list'>, {3100: ['3-0', '6-1', '9-0', '7-0', '8-0', '5-5'], 3101: ['3-1', '6-0', '7-0', '19-2', '11-4', '5-1'], 3102: ['6-1', '10-2', '9-1', '16-1', '11-3', '3-1'], 3103: ['9-0', '8-0', '16-2', '17-2', '7-0', '3-0'], 3104: ['2-0', '10-0', '16-3', '12-0', '11-2', '5-4'], 3105: ['6-0', '3-0', '10-2', '11-4', '12-0', '5-2'], 3106: ['10-2', '9-0', '8-0', '20-0', '16-0', '5-3'], 3107: ['9-1', '6-3', '16-1', '11-1', '20-0', '5-0'], 3108: ['10-0', '3-1', '13-1', '11-2', '7-0', '9-5'], 3109: ['1-0', '2-0', '4-0', '3-0', '5-3', '9-1'], 3110: ['1-1', '4-0', '2-0', '9-2', '18-2', '17-2'], 3111: ['5-3', '9-1', '10-0', '8-0', '18-1', '15-1'], 3112: ['4-0', '5-3', '6-1', '9-3', '8-2', '18-0'], 3113: ['8-0', '7-0', '3-0', '6-1', '20-0', '9-4'], 3114: ['2-1', '9-2', '8-1', '21-0', '14-2', '17-1'], 3115: ['5-4', '9-4', '16-0', '4-0', '20-0', '18-2'], 3116: ['9-2', '10-1', '6-0', '8-3', '16-1', '19-0'], 3117: ['5-5', '9-3', '6-3', '14-1', '20-0', '18-1'], 3118: ['8-1', '6-2', '20-0', 

,0,1,2,3,4,5
3100,3-0,6-1,9-0,7-0,8-0,5-5
3101,3-1,6-0,7-0,19-2,11-4,5-1
3102,6-1,10-2,9-1,16-1,11-3,3-1
3103,9-0,8-0,16-2,17-2,7-0,3-0
3104,2-0,10-0,16-3,12-0,11-2,5-4
3105,6-0,3-0,10-2,11-4,12-0,5-2
3106,10-2,9-0,8-0,20-0,16-0,5-3
3107,9-1,6-3,16-1,11-1,20-0,5-0
3108,10-0,3-1,13-1,11-2,7-0,9-5
3109,1-0,2-0,4-0,3-0,5-3,9-1


In [100]:
timetable_students.to_csv('output_timetable_students_itv_drivent.csv', sep=';', header=False)